# fastduck

> A bit of extra usability for duckdb... inspired by fastlite and sqlite_utils

`fastduck` provides some development experience improvements for the standard `duckdb` python API. 

## Install

```sh
pip install fastduck
```

## How to use



~~import fastduck as fuck~~

In [1]:
from fastduck import database

In [2]:
db = database('../data/chinook.duckdb')
db

DuckDBPyConnection (chinook_main)

In [3]:
dt = db.t
dt

(chinook_main) Tables: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track, fd_Customer, todos, tst

You can use this to grab a single table...

In [4]:
artist = dt.Artist
artist

┌──────────┬────────────────────────────────────────────────────────────────────────────────────┐
│ ArtistId │                                        Name                                        │
│  int32   │                                      varchar                                       │
├──────────┼────────────────────────────────────────────────────────────────────────────────────┤
│        1 │ AC/DC                                                                              │
│        2 │ Accept                                                                             │
│        3 │ Aerosmith                                                                          │
│        4 │ Alanis Morissette                                                                  │
│        5 │ Alice In Chains                                                                    │
│        6 │ Antônio Carlos Jobim                                                               │
│        7 │ Apocaly

In [5]:
customer = dt['Customer']
customer

┌────────────┬───────────┬─────────────┬───┬────────────────────┬──────────────────────┬──────────────┐
│ CustomerId │ FirstName │  LastName   │ … │        Fax         │        Email         │ SupportRepId │
│   int32    │  varchar  │   varchar   │   │      varchar       │       varchar        │    int32     │
├────────────┼───────────┼─────────────┼───┼────────────────────┼──────────────────────┼──────────────┤
│          1 │ Luís      │ Gonçalves   │ … │ +55 (12) 3923-5566 │ luisg@embraer.com.br │            3 │
│          2 │ Leonie    │ Köhler      │ … │ NULL               │ leonekohler@surfeu…  │            5 │
│          3 │ François  │ Tremblay    │ … │ NULL               │ ftremblay@gmail.com  │            3 │
│          4 │ Bjørn     │ Hansen      │ … │ NULL               │ bjorn.hansen@yahoo…  │            4 │
│          5 │ František │ Wichterlová │ … │ +420 2 4172 5555   │ frantisekw@jetbrai…  │            4 │
│          6 │ Helena    │ Holý        │ … │ NULL               

... or multiple tables at once:

In [6]:
dt['Artist', 'Album', 'Genre']

[┌──────────┬────────────────────────────────────────────────────────────────────────────────────┐
 │ ArtistId │                                        Name                                        │
 │  int32   │                                      varchar                                       │
 ├──────────┼────────────────────────────────────────────────────────────────────────────────────┤
 │        1 │ AC/DC                                                                              │
 │        2 │ Accept                                                                             │
 │        3 │ Aerosmith                                                                          │
 │        4 │ Alanis Morissette                                                                  │
 │        5 │ Alice In Chains                                                                    │
 │        6 │ Antônio Carlos Jobim                                                               │
 │        

It also provides auto-complete in Jupyter, IPython and nearly any other interactive Python environment:

![Autocomplete in Jupyter](images/autocomplete.png){width=400}

You can check if a table is in the database already:

In [7]:
'Artist' in dt

True

Column work in a similar way to tables, using the `c` property:

In [8]:
ac = artist.c
ac, artist.columns

(│
 └───────────────────────────────────────────────────────────────────────────────────────────────┘
  Columns: ArtistId, Name,
 ['ArtistId', 'Name'])

Auto-complete works for columns too:


![Columns autocomplete in Jupyter](images/columns_complete.png){width=300}

The tables and views of a database got some interesting new attributes....

In [9]:
artist.meta

{'base': DuckDBPyConnection (chinook_main),
 'catalog': 'chinook',
 'schema': 'main',
 'name': 'Artist',
 'type': 'BASE TABLE',
 'comment': None,
 'shape': (275, 2)}

In [10]:
artist.model

[{'name': 'ArtistId',
  'type': 'INTEGER',
  'nullable': False,
  'default': None,
  'pk': True},
 {'name': 'Name',
  'type': 'VARCHAR',
  'nullable': True,
  'default': None,
  'pk': False}]

In [11]:
artist.cls, type(artist.cls)

(fastduck.core.Artist, type)

`duckdb` replacement scans keep working and are wonderful for usage in SQL statements:

In [12]:
db.sql("select * from artist where artist.Name like 'AC/%'")

┌──────────┬─────────┐
│ ArtistId │  Name   │
│  int32   │ varchar │
├──────────┼─────────┤
│        1 │ AC/DC   │
└──────────┴─────────┘

You can view the results of a query as records

In [13]:
db.sql("select * from artist where artist.Name like 'AC/%'").to_recs()

[{'ArtistId': 1, 'Name': 'AC/DC'}]

or as a list of lists

In [14]:
db.sql("select * from artist where artist.Name like 'AC/%'").to_list()

[[1, 'AC/DC']]

And you there is also an alias for `sql` with `to_recs` simply called `q`

In [15]:
db.q("select * from artist where artist.Name like 'AC/%'")

[{'ArtistId': 1, 'Name': 'AC/DC'}]

#### Dataclass support

As we briefly saw, a  `dataclass` type with the names, types and defaults of the table is added to the Relation:

In [16]:
abm = db.t.Album
art = db.t.Artist
acca_sql = f"""
select abm.* 
from abm join art using (ArtistID)
where art.Name like 'AC/%'
"""
acca_dacca = db.q(acca_sql)
acca_dacca

[{'AlbumId': 1,
  'Title': 'For Those About To Rock We Salute You',
  'ArtistId': 1},
 {'AlbumId': 4, 'Title': 'Let There Be Rock', 'ArtistId': 1}]

In [17]:
let_b_rock_obj = abm.cls(**acca_dacca[-1])
let_b_rock_obj

Album(AlbumId=4, Title='Let There Be Rock', ArtistId=1)

You can get the definition of the dataclass using fastcore's `dataclass_src` -- everything is treated as nullable, in order to handle auto-generated database values:

In [18]:
from fastcore.xtras import hl_md, dataclass_src

src = dataclass_src(db.t.Album.cls)
hl_md(src, 'python')

```python
@dataclass
class Album:
    AlbumId: int32 = None
    Title: str = None
    ArtistId: int32 = None

```